In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
import io

#from google.colab import drive
from sklearn.metrics import confusion_matrix
from keras.models import model_from_json

In [14]:
import os
import glob

#aggregating collected data
paths = glob.glob('./../../data/BTC/BTC*.zip')

full_dataset = None

for i in range(len(paths)):
    if full_dataset is None:
        full_dataset = pd.read_csv('./../../data/BTC/BTC_Book_' + str(i) + '.zip', 
                                   compression = 'zip', index_col = 0, parse_dates = True)
    else:
        app_data = pd.read_csv('./../../data/BTC/BTC_Book_' + str(i) + '.zip', 
                               compression = 'zip', index_col = 0, parse_dates = True)
        full_dataset = full_dataset.append(app_data)
        
full_dataset.head(1)        

,Sequence,Bid_1_Price,Bid_1_Size,Bid_1_Trades,Bid_2_Price,Bid_2_Size,Bid_2_Trades,Bid_3_Price,Bid_3_Size,Bid_3_Trades,...,Ask_47_Trades,Ask_48_Price,Ask_48_Size,Ask_48_Trades,Ask_49_Price,Ask_49_Size,Ask_49_Trades,Ask_50_Price,Ask_50_Size,Ask_50_Trades
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-04-03 17:06:34.424123-04:00,13321475728,6708.69,0.44705,1,6705.57,0.433865,1,6705.56,3.8,1,...,1,6741.24,1.334,1,6743.0,0.058299,1,6743.06,0.927,1


In [15]:
yt = np.load('test_target.npy')
yp = np.load('test_pred.npy')

In [22]:
mid_price = pd.DataFrame((full_dataset['Bid_1_Price'] + full_dataset['Ask_1_Price'])/2, columns = ['mid_price'])
mid_price['target'] = mid_price['mid_price'].diff(1).shift(-1)
sd = mid_price['target'].std()

def return_cat(x, stdev):
    if x > (0 + stdev):
        return 1
    if x < (0 - stdev):
        return -1
    else:
        return 0

mid_price['target_cat'] = mid_price['target'].apply(return_cat, args = (sd,)) #not NAN for indices 0 - 49998 - last is nan
mid_price = mid_price.dropna()

In [43]:
data = mid_price['target_cat'][750000:].values + 1
np.equal(data[29:len(data)], yt[0:len(yt)]).astype(int).sum()

159336

In [49]:
export = pd.DataFrame(mid_price[750029:].values, columns = ['mid_price', 'target', 'target_cat'], 
                      index = mid_price.index[750029:])
export['target_cat'] = export['target_cat'].values + 1

In [50]:
export['Matched_targets'] = yt
export['Predicted_targets'] = yp
export

,mid_price,target,target_cat,Matched_targets,Predicted_targets
Timestamp,,,,,
2020-04-12 09:48:03.412588-04:00,6918.610,0.465,1.0,1,2
2020-04-12 09:48:04.414099-04:00,6919.075,0.000,1.0,1,1
2020-04-12 09:48:05.425153-04:00,6919.075,0.000,1.0,1,2
2020-04-12 09:48:06.410959-04:00,6919.075,0.000,1.0,1,1
2020-04-12 09:48:07.447191-04:00,6919.075,0.000,1.0,1,1
...,...,...,...,...,...
2020-04-14 06:11:30.111359-04:00,6882.055,0.000,1.0,1,2
2020-04-14 06:11:31.101122-04:00,6882.055,0.000,1.0,1,1
2020-04-14 06:11:32.280154-04:00,6882.055,0.000,1.0,1,2


In [51]:
np.equal(export['Matched_targets'].values, export['target_cat'].values)

array([ True,  True,  True, ...,  True,  True,  True])

In [52]:
export.to_csv('test_predictions.csv')